In [1]:
# TODO : File Duplicate of create_transaction_stream.py and will be removed recreated for testing

In [2]:
# Hosted notebook environments may not have the local features package
import os
from IPython import get_ipython


def need_download_modules():
    if 'google.colab' in str(get_ipython()):
        return True
    if 'HOPSWORKS_PROJECT_ID' in os.environ:
        return True
    return False

if need_download_modules():
    print("⚙️ Downloading modules...")
    os.system('mkdir -p synthetic_data')
    os.system('cd synthetic_data && wget https://raw.githubusercontent.com/manu-sj/hopsworks-tutorials/FSTORE-1107/advanced_tutorials/pyspark_streaming/synthetic_data/synthetic_data.py')
    os.system('cd synthetic_data && wget https://raw.githubusercontent.com/manu-sj/hopsworks-tutorials/FSTORE-1107/advanced_tutorials/pyspark_streaming/synthetic_data/create_transaction_stream.py')
    os.system('cd synthetic_data && wget https://raw.githubusercontent.com/manu-sj/hopsworks-tutorials/FSTORE-1107/advanced_tutorials/pyspark_streaming/synthetic_data/init_kafka.py')
    os.system('cd synthetic_data && wget https://raw.githubusercontent.com/manu-sj/hopsworks-tutorials/FSTORE-1107/advanced_tutorials/pyspark_streaming/synthetic_data/__init__.py')
    print('✅ Done!')
else:
    print("Local environment")

Local environment


In [3]:
from synthetic_data import init_kafka
from synthetic_data import synthetic_data
import hsfs
from hsfs.core.storage_connector_api import StorageConnectorApi
from confluent_kafka import Producer

In [ ]:
# creating kafka topic and schema
init_kafka.init() 

# Creating simulated data
data_simulater = synthetic_data.synthetic_data()
credit_cards, trans_df = data_simulater.create_simulated_transactions()
#TODO : Remove and make this into a different dataframe
trans_df = trans_df.drop(["fraud_label"], axis = 1)

In [ ]:
# Getting Kafka Configurations
connection = hsfs.connection()
fs = connection.get_feature_store()
sc_api = StorageConnectorApi()
sc = sc_api.get_kafka_connector(fs.id, False)

In [ ]:
kafka_config = sc.confluent_options()
kafka_config["group.id"] = "test_groupid"

producer = Producer(kafka_config)

for index, transaction in trans_df.iterrows():
    producer.produce(init_kafka.KAFKA_TOPIC_NAME, transaction.to_json())
    producer.flush() # Synchronising read and write blocking call
    if index % 100 == 0 :
        print(f'index {index}')